In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

def kyoutei_data_download(download, url, month):
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {"download.default_directory": download })

    browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    browser.implicitly_wait(10)  # 最大10秒待機

    try:
        # URLへ移動
        browser.get(url)

        for i in range(month):
            # フレームを選択
            browser.switch_to.frame('menu')

            # 月のドロップダウンを選択
            dropdown = browser.find_element(By.NAME, 'MONTH')
            select = Select(dropdown)
            select.select_by_index(i + 1 + 2)#####

            # フレームをリセット
            browser.switch_to.default_content()
            time.sleep(1)

            # "JYOU" フレームに切り替え
            browser.switch_to.frame("JYOU")

            # ラジオボタンを取得しクリックしてダウンロード
            radio_buttons = browser.find_elements(By.XPATH, "//*[@type='radio']")
            for j, radio in enumerate(radio_buttons):
                radio.click()
                time.sleep(1)  # 適度な間隔
                download_button = browser.find_element(By.XPATH, "//*[@value='ダウンロード開始']")
                download_button.click()
                time.sleep(2)

            # フレームをリセット
            browser.switch_to.default_content()
            time.sleep(2)

    except Exception as e:
        print(f"エラーが発生しました: {e}")
        traceback.print_exc()  # エラーの詳細を表示

    finally:
        # ブラウザを閉じる
        browser.quit()

In [2]:
# 必要なモジュールのインポート
import re
import lhafile
import os

def data_kaitou(kyoutei_download_folder,kaitou_folder):
    os.makedirs(kaitou_folder, exist_ok = True)
    lzh_file_list = os.listdir(kyoutei_download_folder)
    for lzh_file_name in lzh_file_list:

        file = lhafile.Lhafile(os.path.join(kyoutei_download_folder, lzh_file_name))

        # 解凍したファイルの名前を取得
        info = file.infolist()
        name = info[0].filename

        # 解凍したファイルの保存
        open(os.path.join(kaitou_folder ,name), "wb").write(file.read(name))



In [3]:
if __name__ == '__main__':
    #必要な項目の入力
    download_folder = r"C:\Users\archi\boatrace\K"
    kaitou_folder = r"C:\Users\archi\boatrace\kaitou_K"
    url = "http://www1.mbrace.or.jp/od2/K/dindex.html"
    manth = 10

    #データのダウンロード処理
    kyoutei_data_download(download_folder, url, manth)

    #圧縮ファイルの解凍処理
    data_kaitou(download_folder, kaitou_folder)

    print('すべての処理が完了しました')

エラーが発生しました: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.166)
Stacktrace:
	GetHandleVerifier [0x008EC7F3+24435]
	(No symbol) [0x00872074]
	(No symbol) [0x007406E3]
	(No symbol) [0x0071F83E]
	(No symbol) [0x007B455E]
	(No symbol) [0x007CEB19]
	(No symbol) [0x007AD5B6]
	(No symbol) [0x0077C54F]
	(No symbol) [0x0077D894]
	GetHandleVerifier [0x00BF70A3+3213347]
	GetHandleVerifier [0x00C0B0C9+3295305]
	GetHandleVerifier [0x00C0558C+3271948]
	GetHandleVerifier [0x00987360+658144]
	(No symbol) [0x0087B27D]
	(No symbol) [0x00878208]
	(No symbol) [0x008783A9]
	(No symbol) [0x0086AAC0]
	BaseThreadInitThunk [0x765C5D49+25]
	RtlInitializeExceptionChain [0x7788CE3B+107]
	RtlGetAppContainerNamedObjectPath [0x7788CDC1+561]



NameError: name 'traceback' is not defined

In [36]:
if __name__ == '__main__':
    #必要な項目の入力
    download_folder = r"C:\Users\archi\boatrace\B"
    kaitou_folder = r"C:\Users\archi\boatrace\kaitou_B"
    url = "http://www1.mbrace.or.jp/od2/B/dindex.html"
    manth = 12

    #データのダウンロード処理
    kyoutei_data_download(download_folder, url, manth)

    #圧縮ファイルの解凍処理
    data_kaitou(download_folder, kaitou_folder)

    print('すべての処理が完了しました')

KeyboardInterrupt: 

In [63]:
import pandas as pd
import re

def parse_boatrace_txt(file_path):
    # Shift_JIS でファイルを開く
    with open(file_path, "r", encoding="shift_jis") as f:
        lines = f.readlines()

    data = []
    race_info = None
    current_race = []
    race_place = None  # レース場情報

    # 空白を削除
    for line in lines:
        line = line.strip()

        # レース場情報を取得（ボートレースの後に3文字を固定取得）
        match_place = re.search(r"ボートレース(.{3})", line)
        
        # レース情報の取得
        match_race = re.match(r"(\d+)Ｒ\s+(.+)\s+Ｈ\d+ｍ\s+電話投票締切予定(\d{2}：\d{2})", line)
        if match_race:
            if current_race:
                data.extend(current_race)
                current_race = []

            race_number = match_race.group(1)
            race_type = match_race.group(2)
            closing_time = match_race.group(3)
            race_info = (race_number, race_type, closing_time, race_place)  # レース場を追加

        # 選手情報の取得（1〜6番目の艇の情報）
        elif re.match(r"^\d\s\d{4}", line):
            parts = line.split(maxsplit=2)  # 先頭2つ（艇番, 登録番号）は固定なので分割
            
            boat_number = parts[0]  # 艇番
            raw_data = parts[1] + " " + parts[2]  # 登録番号以降のデータを結合して処理
            
            try:
                # 登録番号 (4桁固定)
                register_number = raw_data[:4].strip()
                # 選手名 (4文字固定)
                name = raw_data[4:8].strip()
                # 年齢 (2桁固定)
                age = raw_data[8:10].strip()
                # 支部 (2文字固定)
                region = raw_data[10:12].strip()
                # 体重 (2桁固定)
                weight = raw_data[12:14].strip()
                # 級別 (A1, A2, B1, B2)
                rank = raw_data[14:16].strip()
                
                # 残りのデータ（勝率など）を適切に分割
                remaining_data = raw_data[16:].split()

                # `remaining_data` の要素数を確認し、不足している場合は補完
                while len(remaining_data) < 11:
                    remaining_data.append(None)  # データが足りない場合は `None` を補完

                # None を空文字に変換
                remaining_data = [str(x) if x is not None else "" for x in remaining_data]

                win_rate_national = remaining_data[0]
                win_rate_national_2 = remaining_data[1]
                win_rate_local = remaining_data[2]
                win_rate_local_2 = remaining_data[3]
                motor_no = remaining_data[4]
                motor_rate = remaining_data[5]
                boat_no = remaining_data[6]
                boat_rate = remaining_data[7]
                race_performance = " ".join(remaining_data[8:])

                current_race.append([
                    race_info[0], race_info[1], race_info[2], race_info[3],  # レース場を含める
                    boat_number, register_number, name, age, region, weight, rank,
                    win_rate_national, win_rate_national_2, win_rate_local, win_rate_local_2,
                    motor_no, motor_rate, boat_no, boat_rate, race_performance
                ])
            except IndexError:
                print(f"データが不足している行を発見: {line}")

    # 最後のレースデータを追加
    if current_race:
        data.extend(current_race)

    # DataFrame に変換
    columns = [
        "レース番号", "レース種別", "締切時間", "レース場",  # レース場を追加
        "艇番", "登録番号", "選手名", "年齢", "所属", "体重", "級別",
        "全国勝率", "全国2連対率", "当地勝率", "当地2連対率",
        "モーター番号", "モーター2連対率", "ボート番号", "ボート2連対率", "今節成績"
    ]
    
    df = pd.DataFrame(data, columns=columns)
    
    return df

# テキストファイルを変換（Shift_JISで読み込む）
file_path = r"C:\Users\archi\boatrace\kaitou_B\B250217.TXT"  # テキストファイルのパス
df = parse_boatrace_txt(file_path)

# CSVとして保存（Shift_JISで書き出し）
csv_path = "boatrace_data_with_place.csv"
df.to_csv(csv_path, index=False, encoding="shift_jis")

# データの最初の5行を表示
print("データの最初の5行:")
print(df.head())

print(f"\nCSVファイルが保存されました: {csv_path}")


データの最初の5行:
  レース番号            レース種別   締切時間 レース場 艇番  登録番号   選手名  年齢  所属  体重  級別  全国勝率  \
0     １  サンライズＶ           ０８：４３  芦　屋  1  4515  藤田浩人  38  佐賀  54  A2  6.00   
1     １  サンライズＶ           ０８：４３  芦　屋  2  3730  塩崎優司  52  愛知  52  B1  4.72   
2     １  サンライズＶ           ０８：４３  芦　屋  3  4129  宇野博之  44  愛知  55  B1  5.05   
3     １  サンライズＶ           ０８：４３  芦　屋  4  3224  栗山繁洋  59  佐賀  54  B1  4.41   
4     １  サンライズＶ           ０８：４３  芦　屋  5  4111  奥田 誠  44  広島  57  B1  5.09   

  全国2連対率  当地勝率 当地2連対率 モーター番号 モーター2連対率 ボート番号 ボート2連対率        今節成績  
0  43.56  5.69  34.33     16    31.69    44   31.94  423265 10   
1  23.76  2.50   0.00     30    39.11    23   37.37    245 26 8  
2  32.74  2.88   0.00     14    41.27    22   37.50     6 465 7  
3  27.08  4.66  26.32     53    30.00    46   28.72    354 14 5  
4  25.45  4.50  31.25     45    37.63    57   36.11    444 53 6  

CSVファイルが保存されました: boatrace_data_with_place.csv


In [95]:
import pandas as pd
import re
import numpy as np
import datetime

def parse_boatrace_results(file_path):
    with open(file_path, "r", encoding="shift_jis") as f:
        lines = f.readlines()

    race_data = []
    race_info = None
    race_place = None
    race_started = False  # レースデータが始まったかを判定するフラグ

    for i, line in enumerate(lines):
        line = line.strip()

        # 以下のパターンに当てはまる場合はスキップ（関係ないデータ）
        if not line or \
           "払戻金" in line or "単勝" in line or "複勝" in line or "連単" in line or "連複" in line or "人気" in line or \
           "－－－－" in line or "-" * 10 in line or "KBGN" in line or "KEND" in line or "FINALK" in line or \
           "データは、この場の全レース終了後に登録されます。" in line or \
           re.match(r"S\d", line) or re.match(r"K\d", line) or \
           "＊＊＊" in line or "成績" in line or "争奪" in line or "ボートレース" in line or \
           "内容については主催者発行のものと照合して下さい" in line or \
           re.match(r"^第?\d+回", line) or "記念競走" in line:
            continue  # 不要なデータをスキップ

        # 払戻金の行（例: "1R 1-3-5 2910 1-3-5 1580 ...」）をスキップ
        if re.match(r"\d+R\s+\d+-\d+-\d+", line):
            continue

        # レース場の取得（ボートレースXXX の3文字を取得）
        match_place = re.search(r"ボートレース(.{3})", line)
        if match_place:
            race_place = match_place.group(1)

        # レース情報の取得（例: "1R サンライズV H1800m 晴 風 西 6m 波 6cm"）
        match_race = re.match(r"(\d+)R\s+(.+?)\s+H\d+m\s+(\S+)\s+風\s+(\S+)\s+(\d+)m\s+波\s+(\d+)cm", line)
        if match_race:
            race_info = {
                "レース場": race_place,
                "レース番号": match_race.group(1),
                "レース種別": match_race.group(2),
                "天候": match_race.group(3),
                "風向": match_race.group(4),
                "風速(m)": match_race.group(5),
                "波高(cm)": match_race.group(6),
            }
            race_started = True  # ここからレースデータが始まる
            continue

        # レース結果データが始まるまではスキップ
        if not race_started:
            continue

        # 「不成立」や「中止」などの行はスキップ
        if "不成立" in line or "中止" in line:
            continue

        # 競走成績の取得（「着 艇 登番」行の次からデータを取得）
        if re.match(r"^着\s+艇\s+登番", line):
            continue  # この行はスキップ

        # 余分なスペースを適切に処理
        line = re.sub(r"\s+", " ", line)

        # 選手名を正しく取得するための正規表現（名前の途中のスペースを許容）
        match_result = re.match(
            r"([F\d]+) (\d) (\d{4}) ([\u4E00-\u9FAF\s]+) (\d{1,2}) (\d{1,2}) ([\d.]+) (\d) ([F.0-9]+) ([\d.]+|\.)",
            line
        )

        if match_result and race_info:
            # 着順が "F" かどうかを確認し、データを正しく取得
            placement = match_result.group(1)
            
            # 選手名のスペースを削除（姓と名の間のスペースも含む）
            player_name = match_result.group(4).replace(" ", "")

            # レースタイムの処理
            race_time = match_result.group(10)
            if race_time in [".", ". ."]:
                race_time = np.nan

            race_data.append([
                race_info["レース場"], race_info["レース番号"], race_info["レース種別"], 
                race_info["天候"], race_info["風向"], race_info["風速(m)"], race_info["波高(cm)"], 
                placement, match_result.group(2), match_result.group(3), 
                player_name, match_result.group(5), match_result.group(6), 
                match_result.group(7), match_result.group(8), match_result.group(9), 
                race_time
            ])
        else:
            # レースデータが始まっている場合のみエラーメッセージを出す
            if race_started:
                print(f"データ取得ミス（{i + 1}行目）: {line}")

    # DataFrame 変換
    columns = ["レース場", "レース番号", "レース種別", "天候", "風向", "風速(m)", "波高(cm)",
               "着順", "艇番", "登録番号", "選手名", "モーター", "ボート", "展示タイム", 
               "進入", "スタートタイミング", "レースタイム"]

    df = pd.DataFrame(race_data, columns=columns)

    return df

# テキストファイルを変換（Shift_JISで読み込む）
file_path = r"C:\Users\archi\boatrace\kaitou_K\K250217.TXT" 
df_race = parse_boatrace_results(file_path)

# CSVの保存
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
csv_path = f"boatrace_results_{timestamp}.csv"
df_race.to_csv(csv_path, index=False, encoding="shift_jis")

print("【競走成績データ】")
print(df_race.head())
print(f"\nCSVファイルが保存されました: {csv_path}")


データ取得ミス（267行目）: 05 2 3841 佐々木 和 伸 56 17 6.88 2 0.09 . .
データ取得ミス（601行目）: 3-1-6 6180 1-3-6 1000
データ取得ミス（654行目）: 04 6 5326 三 嶌 さらら 19 54 7.16 5 0.30 1.18.0
データ取得ミス（974行目）: 03 6 4560 庄 司 樹良々 56 49 6.91 6 0.20 2.01.7
データ取得ミス（1036行目）: 02 3 3909 佐々木 康 幸 37 54 6.87 3 0.04 1.51.9
データ取得ミス（1119行目）: 01 1 3909 佐々木 康 幸 37 54 6.76 1 0.04 1.13.2
データ取得ミス（1141行目）: 02 3 4560 庄 司 樹良々 56 49 6.84 3 0.26 1.13.7
データ取得ミス（1191行目）: 03 4 5156 坂 野 さくら 42 21 6.82 4 0.19 1.54.6
データ取得ミス（1278行目）: 06 3 3704 本 部 めぐみ 54 62 6.78 3 0.23 . .
データ取得ミス（1360行目）: 04 6 5156 坂 野 さくら 42 21 6.81 6 0.30 1.16.3
データ取得ミス（1485行目）: 11R 中 止
データ取得ミス（1486行目）: 12R 中 止
【競走成績データ】
   レース場 レース番号   レース種別 天候 風向 風速(m) 波高(cm)  着順 艇番  登録番号   選手名 モーター ボート 展示タイム 進入  \
0  None     1  サンライズＶ  晴  西     6      6  01  3  4129  宇野博之   14  22  6.95  3   
1  None     1  サンライズＶ  晴  西     6      6  02  1  4515  藤田浩人   16  44  6.92  1   
2  None     1  サンライズＶ  晴  西     6      6  03  5  4111   奥田誠   45  57  6.93  5   
3  None     1  サンライズＶ  晴  西     6      6  04  2

In [ ]:
r"C:\Users\archi\boatrace\kaitou_K\K250217.TXT" 

In [143]:
import re
import pandas as pd

def parse_boatrace_txt(file_path):
    # Shift_JIS でファイルを開く
    with open(file_path, "r", encoding="shift_jis") as f:
        lines = f.readlines()
    
    race_data = []
    race_info = None
    race_place = None
    race_player = None
    current_race = []
    
    for line in lines:
        line = line.strip()
        if not line:  # 空行はスキップ
            continue
    
        # レース場情報を取得（ボートレースの後に3文字を固定取得）
        match_place = re.search(r"ボートレース(.{3})", line)
        if match_place:
            race_place = match_place.group(1)
        
        # レース情報の取得
        match_race = re.match(r"([０-９])Ｒ\s+(\S+)", line)
        if match_race:
            if current_race:
                race_data.extend(current_race)
                current_race = []
    
            race_number = match_race.group(1)
            race_type = match_race.group(2)
            race_info = (race_number, race_type, race_place)
    
        # 選手情報の取得
        match_player = re.match(r"^\d\s\d{4}", line)
        if match_player:
            parts = line.split(maxsplit=2)  # 先頭2つ（艇番, 登録番号）は固定なので分割
            
            boat_number = parts[0]  # 艇番
            raw_data = parts[1] + " " + parts[2]  # 登録番号以降のデータを結合して処理

            # 登録番号 (4桁固定)
            register_number = raw_data[:4].strip()
            # 選手名 (4文字固定)
            name = raw_data[4:8].strip()
            # 年齢 (2桁固定)
            age = raw_data[8:10].strip()
            # 支部 (2文字固定)
            region = raw_data[10:12].strip()
            # 体重 (2桁固定)
            weight = raw_data[12:14].strip()
            # 級別 (A1, A2, B1, B2)
            rank = raw_data[14:16].strip()
            
            # 残りのデータを適切に分割
            remaining_data = raw_data[16:].split()

            # None を空文字に変換
            remaining_data = [str(x) if x is not None else "" for x in remaining_data]

            win_rate_national = remaining_data[0]
            win_rate_national_2 = remaining_data[1]
            win_rate_local = remaining_data[2]
            win_rate_local_2 = remaining_data[3]
            motor_no = remaining_data[4]
            motor_rate = remaining_data[5]
            boat_no = remaining_data[6]
            boat_rate = remaining_data[7]

            current_race.append([
                race_info[0], race_info[1], race_info[2],
                boat_number, register_number, name, age, region, weight, rank,
                win_rate_national, win_rate_national_2, win_rate_local, win_rate_local_2,
                motor_no, motor_rate, boat_no, boat_rate
            ])
    
    # 最後のレースデータを追加
    if current_race:
        race_data.extend(current_race)
    
    # DataFrame に変換
    columns = [
        "レース番号", "レース種別", "レース場", 
        "艇番", "登録番号", "選手名", "年齢", "所属", "体重", "級別",
        "全国勝率", "全国2連対率", "当地勝率", "当地2連対率",
        "モーター番号", "モーター2連対率", "ボート番号", "ボート2連対率"
    ]

    df = pd.DataFrame(race_data, columns=columns)
    return df

file_path = r"C:\Users\archi\boatrace\kaitou_B\B250201.TXT"  # テキストファイルのパス
df = parse_boatrace_txt(file_path)

# CSVとして保存（Shift_JISで書き出し）
csv_path = "boatrace_data_with_place1.csv"
df.to_csv(csv_path, index=False, encoding="shift_jis")

# データの最初の5行を表示
print("データの最初の5行:")
print(df.head())

print(f"\nCSVファイルが保存されました: {csv_path}")

データの最初の5行:
  レース番号 レース種別 レース場 艇番  登録番号   選手名  年齢  所属  体重  級別  全国勝率 全国2連対率  当地勝率 当地2連対率  \
0     １    一般  大　村  1  4302  西野翔太  41  広島  52  A2  6.67  50.00  6.71  53.85   
1     １    一般  大　村  2  4066  東本勝利  45  三重  54  A1  6.79  51.88  6.95  52.63   
2     １    一般  大　村  3  3489  大場 敏  57  静岡  52  A2  6.24  45.31  6.19  38.71   
3     １    一般  大　村  4  4236  松村 敏  41  福岡  53  A1  7.17  57.80  7.07  58.33   
4     １    一般  大　村  5  3940  飯山 泰  46  東京  52  A2  5.83  39.37  3.80  13.33   

  モーター番号 モーター2連対率 ボート番号 ボート2連対率  
0     72    41.35    45   35.29  
1     50    28.91    32   32.14  
2     56    31.82    36   40.71  
3     75    34.78    70   37.60  
4     54    33.33    56   34.07  

CSVファイルが保存されました: boatrace_data_with_place1.csv


In [129]:
import re
import pandas as pd
def parse_boatrace_results(file_path):
    with open(file_path, "r", encoding="shift_jis") as f:
        lines = f.readlines()
    
    race_data = []
    race_info = None
    race_place = None
    race_player = None
    
    for line in lines:
        line = line.strip()
    
        # レース場の取得
        match_place = re.search(r"(.{1}　.{1})［", line)
        if match_place:
            race_place = match_place.group(1)
    
        # レース情報の取得
        match_race = re.match(r"([0-9]+R)\s+(\S+).*?H\d+m\s+(\S+)\s+風\s+(\S+)\s+(\d+m)\s+波\s+(\d+cm)", line)
        if match_race:
            race_info = {
                "レース場": race_place,
                "レース番号": match_race.group(1),
                "レース種別": match_race.group(2),
                "天候": match_race.group(3),
                "風向": match_race.group(4),
                "風速(m)": match_race.group(5),
                "波高(cm)": match_race.group(6),
                "決まり手": "",
            }
    
        # 決まり手の取得
        match_kimarite = re.search(r"ﾚｰｽﾀｲﾑ\s+(.+)", line)
        if match_kimarite and race_info:
            race_info["決まり手"] = match_kimarite.group(1).replace("　", "").strip()
    
        # 選手データの取得
        match_player = re.match(r"^\d{2}\s+\d\s+\d{4}", line)
        if match_player and race_info:
            race_player = {
                "着順": line[0:4].strip(),
                "艇番": line[4:6].strip(),
                "登録番号": line[6:11].strip(),
                "選手名": line[11:19].replace("　", "").strip(),
                "モーター": line[19:22].strip(),
                "ボート": line[22:27].strip(),
                "展示タイム": line[27:33].strip(),
                "進入": line[33:37].strip(),
                "スタートタイミング": line[37:45].strip(),
                "レースタイム": line[45:56].strip(),
            }
    
            # レース情報と結合して1行分のレースデータを作成
            full_data = {**race_info, **race_player}
            race_data.append(full_data)
    
    # DataFrame 変換
    columns = ["レース場", "レース番号", "レース種別", "天候", "風向", "風速(m)", "波高(cm)", "決まり手",
               "着順", "艇番", "登録番号", "選手名", "モーター", "ボート", "展示タイム", 
               "進入", "スタートタイミング", "レースタイム"]
    
    df = pd.DataFrame(race_data, columns=columns)
    return df

# テキストファイルを変換（Shift_JISで読み込む）
file_path = r"C:\Users\archi\boatrace\kaitou_K\K250217.TXT" 
df_race = parse_boatrace_results(file_path)

# CSVとして保存（Shift_JISで書き出し）
csv_path = "boatrace_results1.csv"
df.to_csv(csv_path, index=False, encoding="shift_jis")

# データの最初の5行を表示
print("データの最初の5行:")
print(df.head())

print(f"\nCSVファイルが保存されました: {csv_path}")

データの最初の5行:
  レース場 レース番号   レース種別 天候 風向 風速(m) 波高(cm) 決まり手  着順 艇番  登録番号   選手名 モーター ボート  \
0  芦　屋    1R  サンライズＶ  晴  西    6m    6cm  まくり  01  3  4129  宇野博之   14  22   
1  芦　屋    1R  サンライズＶ  晴  西    6m    6cm  まくり  02  1  4515  藤田浩人   16  44   
2  芦　屋    1R  サンライズＶ  晴  西    6m    6cm  まくり  03  5  4111   奥田誠   45  57   
3  芦　屋    1R  サンライズＶ  晴  西    6m    6cm  まくり  04  2  3730  塩崎優司   30  23   
4  芦　屋    1R  サンライズＶ  晴  西    6m    6cm  まくり  05  6  3614   谷勝幸   26  64   

  展示タイム 進入 スタートタイミング  レースタイム  
0  6.95  3      0.11  1.50.7  
1  6.92  1      0.10  1.51.2  
2  6.93  5      0.16  1.52.3  
3  7.01  2      0.12  1.53.7  
4  6.92  6      0.15    .  .  

CSVファイルが保存されました: boatrace_results1.csv


In [37]:
file_path = r"C:\Users\archi\boatrace\kaitou_K\K250217.TXT" 

In [6]:
import datetime
datetime.date.today().month

3

In [9]:
# 指定された日付
date_str = "2025-03-14"

# 年、月、日を抽出
year, month, day = map(int,date_str.split('-'))

In [1]:
    def scrape_data_by_month(self, months=12,BorK="B"):
        """今月から指定の月分だけ、スクレイピングを行い、データをダウンロード"""        
        # Chrome options for downloading
        
        browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=self.options)
        browser.implicitly_wait(10)  # 最大10秒待機

        try:
            # URLへ移動
            browser.get(self.url)
    
            for i in range(months):
                # フレームを選択
                browser.switch_to.frame('menu')
    
                # 月のドロップダウンを選択
                dropdown = browser.find_element(By.NAME, 'MONTH')
                select = Select(dropdown)
                select.select_by_index(i+1)
    
                # フレームをリセット
                browser.switch_to.default_content()
                time.sleep(1)
    
                # "JYOU" フレームに切り替え
                browser.switch_to.frame("JYOU")
    
                # ラジオボタンを取得しクリックしてダウンロード
                radio_buttons = browser.find_elements(By.XPATH, "//*[@type='radio']")
                for j, radio in enumerate(radio_buttons):
                    radio.click()
                    time.sleep(1)  # 適度な間隔
                    download_button = browser.find_element(By.XPATH, "//*[@value='ダウンロード開始']")
                    download_button.click()
                    time.sleep(2)
    
                # フレームをリセット
                browser.switch_to.default_content()
                time.sleep(2)

        except Exception as e:
            print(f"エラーが発生しました: {e}")
            traceback.print_exc()  # エラーの詳細を表示
        finally:
            browser.quit()
            

In [2]:
import os
from scraper import BoatraceScraper

# 共通設定
folder = "C:\\Users\\msy-t\\boatrace-ai\\data"
B_url = "https://www1.mbrace.or.jp/od2/B/dindex.html"
K_url = "https://www1.mbrace.or.jp/od2/K/dindex.html"

scraper = BoatraceScraper(folder, BorK="B")
#scraper.scrape_and_process_data_for_date_range(start_date="2025-02-18",end_date="2025-03-27")
df = scraper.parse_B_txt(file_name="B250217")
df

,レース場,レース番号,レース種別,艇番,登録番号,選手名,年齢,支部,体重,級別,全国勝率,全国2連対率,当地勝率,当地2連対率,モーター番号,モーター2連対率,ボート番号,ボート2連対率
0,芦 屋,１,サンライズＶ,1,4515,藤田浩人,38,佐賀,54,A2,6.00,43.56,5.69,34.33,16,31.69,44,31.94
1,芦 屋,１,サンライズＶ,2,3730,塩崎優司,52,愛知,52,B1,4.72,23.76,2.50,0.00,30,39.11,23,37.37
2,芦 屋,１,サンライズＶ,3,4129,宇野博之,44,愛知,55,B1,5.05,32.74,2.88,0.00,14,41.27,22,37.50
3,芦 屋,１,サンライズＶ,4,3224,栗山繁洋,59,佐賀,54,B1,4.41,27.08,4.66,26.32,53,30.00,46,28.72
4,芦 屋,１,サンライズＶ,5,4111,奥田 誠,44,広島,57,B1,5.09,25.45,4.50,31.25,45,37.63,57,36.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,桐 生,９,予選特賞,2,4607,鶴田勇雄,35,福岡,52,A2,5.68,35.29,0.00,0.00,45,50.00,16,22.58
1004,桐 生,９,予選特賞,3,4608,荒井翔伍,35,東京,54,A2,5.70,38.76,6.62,40.54,47,42.11,62,33.33
1005,桐 生,９,予選特賞,4,5112,砂長知輝,25,埼玉,52,A2,6.42,44.95,5.86,37.84,25,52.63,56,36.36
1006,桐 生,９,予選特賞,5,4467,渡邉俊介,38,愛知,51,B1,5.19,32.22,6.17,48.78,40,42.86,37,38.46


In [3]:
scraper = BoatraceScraper(folder, BorK="K")
#scraper.scrape_and_process_data_for_date_range(start_date="2025-02-18",end_date="2025-03-27")
df = scraper.parse_K_txt(file_name="K250217")
df

,レース場,レース番号,レース種別,天候,風向,風速(m),波高(cm),決まり手,着順,艇番,登録番号,選手名,モーター,ボート,展示タイム,進入,スタートタイミング,レースタイム
0,芦 屋,1,サンライズＶ,晴,西,6,6,まくり,01,3,4129,宇野博之,14,22,6.95,3,0.11,1.50.7
1,芦 屋,1,サンライズＶ,晴,西,6,6,まくり,02,1,4515,藤田浩人,16,44,6.92,1,0.10,1.51.2
2,芦 屋,1,サンライズＶ,晴,西,6,6,まくり,03,5,4111,奥田誠,45,57,6.93,5,0.16,1.52.3
3,芦 屋,1,サンライズＶ,晴,西,6,6,まくり,04,2,3730,塩崎優司,30,23,7.01,2,0.12,1.53.7
4,芦 屋,1,サンライズＶ,晴,西,6,6,まくり,05,6,3614,谷勝幸,26,64,6.92,6,0.15,. .
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,常 滑,10,予選特賞,曇り,北,8,10,まくり差し,02,3,4610,佐藤大佑,12,58,6.84,3,0.26,1.57.5
405,常 滑,10,予選特賞,曇り,北,8,10,まくり差し,03,2,4568,廣瀬真也,61,21,6.80,2,0.25,1.58.0
406,常 滑,10,予選特賞,曇り,北,8,10,まくり差し,04,1,5034,若林義人,22,43,6.86,1,0.27,1.59.7
407,常 滑,10,予選特賞,曇り,北,8,10,まくり差し,05,5,4327,加藤翔,62,12,6.77,5,0.37,. .


In [7]:
import pandas as pd

# --- ステップ 1: CSV読み込み（Shift_JIS対応） ---
b_df = pd.read_csv("C:\\Users\\msy-t\\boatrace-ai\\data\\B_csv\\B250327.csv", encoding="shift_jis")
k_df = pd.read_csv("C:\\Users\\msy-t\\boatrace-ai\\data\\K_csv\\K250327.csv", encoding="shift_jis")


# --- ステップ 2: 列名整形と必要列抽出 ---
k_df.columns = k_df.columns.str.strip()
df = k_df[["登録番号", "選手名", "艇番", "決まり手"]].copy()

# --- ステップ 3: 数値変換と決まり手分類関数の定義 ---
df["艇番"] = pd.to_numeric(df["艇番"], errors="coerce")

def classify_kimarite(k):
    if k in ["逃げ"]:
        return "逃げ"
    elif k in ["まくり"]:
        return "まくり"
    elif k in ["差し"]:
        return "差し"
    elif k in ["まくり差し", "まくりざし"]:
        return "まくり差し"
    elif k in ["抜き"]:
        return "抜き"
    elif pd.isna(k):
        return None
    else:
        return "その他"

df["決まり手分類"] = df["決まり手"].apply(classify_kimarite)

# --- ステップ 4: 号艇ごとの決まり手集計 ---
agg_list = []

for boat in range(1, 7):
    temp = df[df["艇番"] == boat]
    pivot = temp.pivot_table(index=["登録番号", "選手名"], 
                             columns="決まり手分類", 
                             values="決まり手", 
                             aggfunc="count", 
                             fill_value=0)
    pivot.columns = [f"{boat}号艇時{col}" for col in pivot.columns]
    agg_list.append(pivot)

# --- ステップ 5: 結合と補完 ---
final_df = pd.concat(agg_list, axis=1).reset_index()

# 欠損補完（出なかった列は0で補う）
all_cols = []
for i in range(1, 7):
    for k in ["逃げ", "まくり", "差し", "まくり差し", "抜き", "その他"]:
        all_cols.append(f"{i}号艇時{k}")
for col in all_cols:
    if col not in final_df.columns:
        final_df[col] = 0

# 列順整備
final_df = final_df[["登録番号", "選手名"] + all_cols]

# --- 🔽 ステップ 6: 登録番号で昇順ソート ---
final_df = final_df.sort_values("登録番号").reset_index(drop=True)

# --- 任意: CSV保存 ---
#final_df.to_csv("Kファイル_決まり手集計_登録番号順.csv", index=False, encoding="utf-8-sig")
final_df


,登録番号,選手名,1号艇時逃げ,1号艇時まくり,1号艇時差し,1号艇時まくり差し,1号艇時抜き,1号艇時その他,2号艇時逃げ,2号艇時まくり,...,5号艇時差し,5号艇時まくり差し,5号艇時抜き,5号艇時その他,6号艇時逃げ,6号艇時まくり,6号艇時差し,6号艇時まくり差し,6号艇時抜き,6号艇時その他
0,3054,北川幸典,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,3075,中村裕将,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0
2,3081,岡本慎治,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3159,江口晃生,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3176,長谷川巌,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,5385,鈴木柊介,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0
568,5386,米玉利大悟,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
569,5388,佐野優都,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0
570,5389,恵良琴美,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
import pandas as pd

# --- ステップ1：CSV読み込み ---
df = pd.read_csv("C:\\Users\\msy-t\\boatrace-ai\\data\\K_csv\\K250327.csv", encoding="shift_jis")

# フラグを作成し、整数に変換
df["1着"] = df["着順"] == 1
df["2着"] = df["着順"] == 2
df["3着"] = df["着順"] == 3
decision_types = ["逃げ", "差し", "まくり", "まくり差し", "抜き", "恵まれ"]
for k in decision_types:
    df[k] = (df["決まり手"] == k).astype(int)
    
# 集計処理
agg_columns = {
    "着順": "count",
    "1着": "sum",
    "2着": "sum",
    "3着": "sum",
    "スタートタイミング": "mean"
}
for k in decision_types:
    agg_columns[k] = "sum"
# 集計実行andカラム名設定    
result = df.groupby(["登録番号", "選手名", "艇番"]).agg(agg_columns).reset_index()
result.columns = ["登録番号", "選手名", "艇番", "出走数", "勝利回数", "2着回数", "3着回数", "平均ST"] + decision_types

# 集計結果を表示
result

,登録番号,選手名,艇番,出走数,勝利回数,2着回数,3着回数,平均ST,逃げ,差し,まくり,まくり差し,抜き,恵まれ
0,3054,北川幸典,2,1,0,1,0,0.04,1,0,0,0,0,0
1,3054,北川幸典,5,1,1,0,0,0.04,0,0,1,0,0,0
2,3075,中村裕将,3,1,0,0,0,0.14,1,0,0,0,0,0
3,3075,中村裕将,6,1,0,0,0,0.13,1,0,0,0,0,0
4,3081,岡本慎治,3,1,0,1,0,0.22,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,5388,佐野優都,6,2,0,0,2,0.08,0,0,1,1,0,0
891,5389,恵良琴美,5,1,0,0,0,0.16,1,0,0,0,0,0
892,5389,恵良琴美,6,1,0,0,0,0.37,0,0,0,1,0,0
893,5391,出穂和鼓,5,1,0,0,0,0.16,0,1,0,0,0,0


In [ ]:
import pandas as pd
import os
from datetime import datetime

# --- ステップ1：ファイル読み込み関数 ---
def load_and_process_file(file_path):
    df = pd.read_csv(file_path, encoding="shift_jis")
    
    # フラグを作成し、整数に変換
    df["1着"] = (df["着順"] == 1).astype(int)
    df["2着"] = (df["着順"] == 2).astype(int)
    df["3着"] = (df["着順"] == 3).astype(int)
    
    decision_types = ["逃げ", "差し", "まくり", "まくり差し", "抜き", "恵まれ"]
    for k in decision_types:
        df[k] = (df["決まり手"] == k).astype(int)
    
    return df

# --- ステップ2：指定範囲のファイルを読み込み ---
def load_files_in_date_range(folder_path, start_date, end_date):
    all_files = os.listdir(folder_path)
    relevant_files = []

    for file in all_files:
        if file.startswith("K") and file.endswith(".csv"):
            try:
                # 日付部分を抽出してチェック
                file_date = datetime.strptime(file[1:9], "%y%m%d")
                if start_date <= file_date <= end_date:
                    relevant_files.append(os.path.join(folder_path, file))
            except ValueError:
                continue
    
    return relevant_files

# --- ステップ3：集計処理 ---
def aggregate_data(files):
    df_list = [load_and_process_file(file) for file in files]
    combined_df = pd.concat(df_list, ignore_index=True)
    
    agg_columns = {
        "着順": "count",
        "1着": "sum",
        "2着": "sum",
        "3着": "sum",
        "スタートタイミング": "mean",
        "逃げ": "sum",
        "差し": "sum",
        "まくり": "sum",
        "まくり差し": "sum",
        "抜き": "sum",
        "恵まれ": "sum"
    }

    result = combined_df.groupby(["登録番号", "選手名", "艇番"]).agg(agg_columns).reset_index()
    result.columns = ["登録番号", "選手名", "艇番", "出走数", "勝利回数", "2着回数", "3着回数", "平均ST", 
                      "逃げ", "差し", "まくり", "まくり差し", "抜き", "恵まれ"]
    
    return result

# --- ステップ4：日付範囲の指定と実行 ---
folder_path = "C:\\Users\\msy-t\\boatrace-ai\\data\\K_csv\\"  # あなたのCSVフォルダのパス
start_date = datetime(2025, 3, 1)
end_date = datetime(2025, 3, 27)

files = load_files_in_date_range(folder_path, start_date, end_date)
result = aggregate_data(files)



In [3]:
from analyzer import BoatraceAnalyzer

analyzer = BoatraceAnalyzer(folder = "C:\\Users\\msy-t\\boatrace-ai\\data")
result = analyzer.aggregate_data(start_date="2024-03-01", end_date="2025-03-27")
result.to_csv("C:\\Users\\msy-t\\boatrace-ai\\data\\aggregated_result.csv", index=False, encoding="shift_jis")
result

,登録番号,選手名,艇番,出走数,勝利回数,2着回数,3着回数,平均ST,逃げ,差し,まくり,まくり差し,抜き,恵まれ
0,2014,高塚清一,1,27,3,10,2,0.158148,3,9,8,2,4,1
1,2014,高塚清一,2,23,2,4,4,0.185652,11,5,5,2,0,0
2,2014,高塚清一,3,27,0,5,3,0.198519,15,5,0,4,3,0
3,2014,高塚清一,4,24,3,3,3,0.178333,8,2,6,2,5,1
4,2014,高塚清一,5,25,0,3,2,0.187200,15,2,5,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9509,5390,高橋涼夏,5,14,0,0,0,0.206429,6,3,2,3,0,0
9510,5390,高橋涼夏,6,32,0,0,0,0.199375,13,3,9,3,2,2
9511,5391,出穂和鼓,4,1,0,0,0,0.150000,0,1,0,0,0,0
9512,5391,出穂和鼓,5,19,0,0,0,0.208421,7,4,6,2,0,0
